Attempt to use the camera to send individual robots to specific x,y points

Also, test out the new SpheroTeam Library!





In [1]:
# Foundational Sphero API, forked from Simon's Library
import sphero

In [2]:
# Tracking code
import cv2

# SpheroNav base classes
from tracker.trackingfilter import FilterSpheroBlueCover, FilterGlow
from tracker.traceable import TraceableObject
from tracker.trackerbase import ColorTracker
from util import Vector2D

In [3]:
# My first python module!
import SpheroTeam

from SpheroTeam.teamutil import readJsonFile, normalize_angle, angle_between_points
from SpheroTeam.navigation import calibrate_bot_direction, get_bot_position
from SpheroTeam.formations import roll_polygon

In [4]:
# General Python Imports
import time
import os

#### SpheroTeam Setup


For more details about what each line does, refer to the previous notebook (6)


In [5]:
# This is a list of the colored "initial names" of the spheros you want to work with in this session
ACTIVEBOTS = ['GRY']

In [6]:
PROJ_ROOT = os.getcwd()
CONFIG = os.path.join(PROJ_ROOT, "config.json")
manager = sphero.SpheroManager()

# Initialize Sphero manager using options provided in the config file
manager= SpheroTeam.initialize.load_sphero_roster(manager, CONFIG)

# Connect to robots
bots = SpheroTeam.initialize.connect_sphero_team(manager, ACTIVEBOTS)

# Check power, robots shut down after dropping below 7 volts
SpheroTeam.print_team_status(bots)

Sphero-GRY Try #0
IOError Occurred. Trying to connect again.
Sphero-GRY Try #1
IOError Occurred. Trying to connect again.
Sphero-GRY Try #2
IOError Occurred. Trying to connect again.
Sphero-GRY Try #3
IOError Occurred. Trying to connect again.
Sphero-GRY Try #4
IOError Occurred. Trying to connect again.
Sphero-GRY Try #5
IOError Occurred. Trying to connect again.
Sphero-GRY Try #6
IOError Occurred. Trying to connect again.
Sphero-GRY Try #7
IOError Occurred. Trying to connect again.
Sphero-GRY Try #8
IOError Occurred. Trying to connect again.
Sphero-GRY Try #9
IOError Occurred. Trying to connect again.
An exception of type SpheroConnectionError occurred. Args:
('Failed to connect after 10 retries. Is the device turned on?',)
Sphero-GRY Try #0
Connected!
Sphero-GRY:Battery OK | 8.0 V


#### Camera Tracker Setup



In [7]:
tracker = ColorTracker()

In [8]:
traceable_blue = TraceableObject("BLUE")
traceable_blue.filter = FilterSpheroBlueCover()

TRACEABLE_COLOR = [0, 0, 255]

In [9]:
# Team Calibration Routine
def get_team_offsets(bots, traceable_object, traceable_color):
    offsets = []
    
    for bot in bots:
        offset = calibrate_bot_direction(bot,  traceable_blue, [0, 0, 255], tracker, True)
        offsets.append(offset)
        proceed = raw_input("'q' to quit, else continue")
        
        if proceed == "q":
            break
        else:
            continue
    return offsets

In [10]:
offsets = get_team_offsets(bots, traceable_blue, TRACEABLE_COLOR)

Start (280,281)
End   (331,219)
Angle 309.440052737
'q' to quit, else continueq


In [11]:
?roll_polygon

In [17]:
# Roll in a square to show it's working
roll_polygon(bots, 1, offsets, 180, speed=70)

Dispatch Time 0.0090000629425


### Lastly: Try for Basic Pid-Free Control

We have a problem because we can't really get sphero's current speed given current latency issues.

Therefore, we will try to move into position using the "scooting" method.


In [48]:
import math

In [124]:
def bot_to_point(bot, offset,
                 targetX, targetY, 
                 TIMEOUT=800,
                 trace_object=traceable_blue, trace_color=TRACEABLE_COLOR, 
                 tracker=tracker, MAX_SECONDS=10, stopRadius=40):
    
    """
        Currently makes use of a few implicit globals for simplicity
        specifically traceable_blue, traceable_color
        Maybe set stop radius  based on pixel size
        
        Wiggles because aiming for direct path / precision
        
        Constrast to "RUSH TO POINT IN STRAIGHT LINE"
        
        Really would work better with a bigger field
        
        Maybe we can push lighter things?
        
        Or boost speed when within range: pause and then burst
    """
    
    currentX, currentY = get_bot_position(bot, trace_object, tracker)

    # Basic closed loop controller
    startTime = time.time()
    
    # Angle to distance
    deltaX = targetX - currentX
    deltaY = targetY - currentY
    angle = math.degrees(math.atan2(deltaY, deltaX))
    distance = math.sqrt(deltaX * deltaX + deltaY * deltaY)
    
    print("Get from {},{} to {},{}| Distance {} / {}").format(currentX, currentY, 
                                                         targetX, targetY, distance, angle)
    bot.set_motion_timeout(TIMEOUT)

    # REPLACE SOMEDAY WITH TRUE PID CONTROLLER
    while distance > stopRadius and (((time.time() - startTime ) < MAX_SECONDS)):
        # these should be tuned based on how many pixels 1 ball is.
        
        if distance < 60:
            print("SUPER CLOSE")
            outSpeed = 33
        elif distance < 100:
            outSpeed = 40
        elif distance < 250:
            outSpeed = 60
        else:
            time.sleep(0.3)
            outSpeed = 70

        # roll the sphero, make use of the request object
        print("Dist {} outSpeed {} at {} degrees: {},{}"\
              .format(distance, outSpeed, angle, currentX, currentY))
        
        SpheroTeam.roll_sphero(bot, outSpeed, -angle, offset)
        
#         bot.roll(outSpeed, angle + offset)
        time.sleep(TIMEOUT/1000.0)
        time.sleep(.3) # give camera time to catch up

        currentX , currentY = get_bot_position(bot, trace_object, tracker)

        # Repeat waypointing calculation
        deltaX = targetX - currentX
        deltaY = targetY - currentY
        
        angle = math.degrees(math.atan2(deltaY, deltaX))
        distance = math.sqrt(deltaX * deltaX + deltaY * deltaY)

    print("Stopped at {},{}, with dist {}").format(currentX, currentY, distance)

In [125]:
bots[0].set_rgb(0, 0, 255)


In [126]:
bot_to_point(bots[0], offsets[0], 350, 350, TIMEOUT=500)

Get from 312,416 to 350,350| Distance 76.1577310586 / -60.0684881595
Dist 76.1577310586 outSpeed 40 at -60.0684881595 degrees: 312,416
Dist 70.7248188403 outSpeed 40 at -46.1457628382 degrees: 301,401
Dist 65.0 outSpeed 40 at -22.619864948 degrees: 290,375
SUPER CLOSE
Dist 56.5685424949 outSpeed 33 at 8.13010235416 degrees: 294,342
SUPER CLOSE
Dist 50.8035431835 outSpeed 33 at 36.1932073056 degrees: 309,320
SUPER CLOSE
Dist 49.3963561409 outSpeed 33 at 68.6293777307 degrees: 332,304
SUPER CLOSE
Dist 45.3982378513 outSpeed 33 at 97.5946433686 degrees: 356,305
SUPER CLOSE
Dist 44.687805943 outSpeed 33 at 130.462227492 degrees: 379,316
SUPER CLOSE
Dist 40.3112887415 outSpeed 33 at 156.614778943 degrees: 387,334
Stopped at 387,352, with dist 37.0540146273


In [127]:
# I think the key is small timeouts
bot_to_point(bots[0], offsets[0], 150, 150, TIMEOUT=400)

bot_to_point(bots[0], offsets[0], 300, 300, TIMEOUT=400)

Get from 388,352 to 150,150| Distance 312.166622175 / -139.677417859
Dist 312.166622175 outSpeed 70 at -139.677417859 degrees: 388,352
Dist 303.534182589 outSpeed 70 at -132.997466868 degrees: 357,372
Dist 280.417545813 outSpeed 70 at -123.066695907 degrees: 303,385
Dist 256.462472888 outSpeed 70 at -112.465197653 degrees: 248,387
Dist 237.305710003 outSpeed 60 at -97.9935214936 degrees: 183,385
Dist 216.335387766 outSpeed 60 at -83.6305607939 degrees: 126,365
Dist 202.249845488 outSpeed 60 at -69.1455419604 degrees: 78,339
Dist 182.78402556 outSpeed 60 at -55.7013507239 degrees: 47,301
Dist 167.107749671 outSpeed 60 at -38.9275435928 degrees: 20,255
Dist 149.485785277 outSpeed 60 at -21.5878858307 degrees: 11,205
Dist 138.177422179 outSpeed 60 at -2.9038188651 degrees: 12,157
Stopped at 27,114, with dist 128.16005618
Get from 28,107 to 300,300| Distance 333.516116552 / 35.3579425797
Dist 333.516116552 outSpeed 70 at 35.3579425797 degrees: 28,107
Dist 309.919344346 outSpeed 70 at 42.90

Maybe bots wiggle around to move into position, and then use blunt force to rush the target??!! 

Anyways this now works with 1 robot!!! WOOHOO

Worry about robots bumping into each other later
